## MNIST nn

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

In [6]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [7]:
# nn layers
linear1 = torch.nn.Linear(784, 256, bias = True)
linear2 = torch.nn.Linear(256, 256, bias = True)
linear3 = torch.nn.Linear(256, 10, bias = True)
relu = torch.nn.ReLU()

In [8]:
# initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[-0.6645,  0.6646,  0.7159,  ..., -0.3040, -0.8945, -0.7977],
        [-1.2045,  2.4545, -1.8073,  ..., -1.7139, -0.0451,  0.3864],
        [ 0.3414,  0.3114, -1.9218,  ..., -0.4525, -0.6849,  0.9663],
        ...,
        [ 1.3915, -0.4048,  0.2338,  ..., -0.8257, -0.8397,  1.9816],
        [ 1.0414,  0.2130, -0.0417,  ...,  1.7541, -0.6454, -0.0821],
        [-0.1162, -1.2692,  0.8201,  ..., -0.8303, -0.5022,  0.0583]],
       requires_grad=True)

In [9]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [11]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        X = X.view(-1, 28 * 28).to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch : ', '%03d' % (epoch + 1), 'Cost = ', '{:.9f}'.format(avg_cost))
    
print('Learning Finished.')

Epoch :  001 Cost =  129.379852295
Epoch :  002 Cost =  36.195644379
Epoch :  003 Cost =  23.019363403
Epoch :  004 Cost =  15.959907532
Epoch :  005 Cost =  11.481480598
Epoch :  006 Cost =  8.563683510
Epoch :  007 Cost =  6.402721882
Epoch :  008 Cost =  4.730027199
Epoch :  009 Cost =  3.540746927
Epoch :  010 Cost =  2.647785187
Epoch :  011 Cost =  2.033876657
Epoch :  012 Cost =  1.644687057
Epoch :  013 Cost =  1.179143310
Epoch :  014 Cost =  1.109077334
Epoch :  015 Cost =  0.785691142
Learning Finished.


In [12]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    pred = model(X_test)
    correct_pred = torch.argmax(pred, 1) == y_test
    accuracy = correct_pred.float().mean()
    print('Accuracy : ', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    y_single_data = mnist_test.test_labels[r:r + 1].to(device)
    
    print('Label : ', y_single_data.item())
    single_pred = model(X_single_data)
    print('Prediction : ', torch.argmax(single_pred, 1).item())

Accuracy :  0.9451000094413757
Label :  9
Prediction :  9


C:\Users\seho\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\seho\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
